# <h1 align = "center"><div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Red Wine Quality: XGBoost + Optuna</div></h1>

<div style="width: 100%; text-align: center;"> <img align = middle src="https://labelyourdata.com/img/article-illustrations/ml_essential_tool.jpg" style = "height: 650px;" width = 100%></div>

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Dataset Fields</div>
> - **Fixed Acidity**: Fixed acidity corresponds to the set of low volatility organic acids such as malic, lactic, tartaric or citric acids and is inherent to the characteristics of the sample.
> - **Volatile Acidity**: Volatile acidity is an important sensory parameter, with higher levels indicating wine spoilage.
> - **Citric Acid**: Citric acid is often added to wines to increase acidity, complement a specific flavor or prevent ferric hazes. It can be added to finished wines to increase acidity and give a *fresh* flavor.
> - **Residual Sugar**: Residual Sugar is from natural grape sugars leftover in a wine after the alcoholic fermentation finishes.
> - **Chlorides**: They are a major contributor to the *saltiness* of a wine.
> - **Free Sulfur Dioxide**: Sulfur dioxide (SO<sub>2</sub>) preserves wine, preventing oxidation and browning.
> - **Total Sulfur Dioxide**: Total Sulfur Dioxide (TSO<sub>2</sub>) is the portion of SO<sub>2</sub> that is free in the wine plus the portion that is bound to other chemicals in the wine such as aldehydes, pigments, or sugars.
> - **Density**: Holding alcohol level constant, density has little effect on the quality of wines as other keys can contribute in density.
> - **pH**: Winemakers use pH as a way to measure ripeness in relation to acidity. Low pH wines will taste tart and crisp, while higher pH wines are more susceptible to bacterial growth.
> - **Sulphates**: The presence of another type of sulpate, is thought to help rid the wine of a wide variety of bacteria (good and bad). This seems to lower the wine quality as well because it dulls the wine's fermentation process.
> - **Alcohol**: Alcohol content affects a wine's body, since alcohol is more viscous than water. A wine with higher alcohol content will have a fuller, richer body, while a lower alcohol wine will taste lighter and more delicate on the palate.
> - **Quality**: Wine quality mainly depends on the vinification process and the geographical origin of the grapes but also highly relies on the varietal composition of the grape.

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Imports</div>


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
import matplotlib.image as mpimg
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from catboost import CatBoostClassifier

from xgboost import XGBClassifier

import lightgbm as lgbm

import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

import warnings
warnings.filterwarnings('ignore')

### Setting a color scheme

In [ ]:
custom_colors = ['#A70000', '#CC0001', '#FF0000', '#FF5252', '#FF7B7B', '#FFBABA']
custom_palette = sns.set_palette(sns.color_palette(custom_colors))
sns.palplot(sns.color_palette(custom_colors), size = 1)
plt.tick_params(axis = 'both', labelsize = 0, length = 0)

### Reading the data

In [ ]:
df = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head()

Taking a look at the missing values of the dataset

In [ ]:
print(df.isna().sum())
print('================================')
print('Total Missing Values = {}'.format(df.isna().sum().sum()))
print('================================')

Taking a look at the statistical summary of the dataset

In [ ]:
summary = pd.DataFrame(df.describe())
summary = summary.style.background_gradient(cmap = 'Reds') \
          .set_table_attributes("style = 'display: inline'") \
          .set_caption('Statistics of the Dataset') \
          .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '16px')
                ]
          }])
summary

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Exploratory Data Analysis (EDA)</div>


In [ ]:
wine_glass = mpimg.imread('../input/plotimages/wawg.jpg')
imagebox = OffsetImage(wine_glass, zoom = 0.5)
xy = (0.5, 0.7)
ab = AnnotationBbox(imagebox, xy, frameon = False, pad = 1, xybox = (4.8, 525))

plt.figure(figsize = (15, 10))
ax = sns.countplot(data = df, x = 'quality', palette = [custom_colors[0], custom_colors[1], custom_colors[2], custom_colors[3], custom_colors[4], custom_colors[5]])
ax.add_artist(ab)

bbox_args = dict(boxstyle = 'round', fc = '0.9')
for p in ax.patches:
        ax.annotate('{:.0f}'.format(p.get_height()), (p.get_x() + 0.3, p.get_height() + 10.5),
                   color = 'black',
                   bbox = bbox_args,
                   fontsize = 15)
        
plt.title('Quality Count for Red Wine', fontsize = 25)
plt.xlabel('Quality', fontsize = 20)
plt.ylabel('Count', fontsize = 20)
plt.xticks(fontsize = 13)
plt.yticks(fontsize = 13)
plt.show()

**Observations**: The count for wine with a quality of **5** is the highest and the count for wine with a quality of **3** is the lowest. From the countplot we can see that there is a **class imbalance** that exists in the dataset, and will need correction before we start training our ML models.

In [ ]:
fig, axes = plt.subplots(ncols = 2, figsize = (30, 10))

sns.histplot(data = df, x = 'citric acid', kde = True, ax = axes[0])
axes[0].set_title('Histogram for Citric Acid', fontsize = 25)
axes[0].set_xlabel('Citric Acid', fontsize = 20)
axes[0].set_ylabel('Count', fontsize = 20)
axes[0].xaxis.set_tick_params(labelsize = 16)
axes[0].yaxis.set_tick_params(labelsize = 16)

sns.histplot(data = df, x = 'alcohol', kde = True, ax = axes[1])
axes[1].set_title('Histogram for Alcohol', fontsize = 25)
axes[1].set_xlabel('Alcohol', fontsize = 20)
axes[1].set_ylabel('Count', fontsize = 20)
axes[1].xaxis.set_tick_params(labelsize = 16)
axes[1].yaxis.set_tick_params(labelsize = 16)

plt.show()

**Observations**: The histograms for `citric acid` and `alcohol` are plotted to take a look at the skewness of their distributions. The histogram for **`alcohol`** exhibits **right skewness** where the `mode < median < mean`. 

Boxplots are a standardized way of displaying the distribution of data. A boxplot is a graph that gives you a good indication of how the values in the data are spread out. Although boxplots may seem primitive in comparison to a histogram or density plot, they have the advantage of taking up less space, which is useful when comparing distributions between many groups or datasets. This is what a typical boxplot would look like:

<img src = "https://phastdata.org/sites/phastdata.org/files/pictures/box_plot_anatomy.png" width = 450>

In [ ]:
plt.figure(figsize = (30, 30))

def create_boxplot(feature):
    sns.boxplot(data = df, x = df['quality'], y = feature)
    plt.title('Box Plot for ' + feature.title(), fontsize = 25)
    plt.xlabel('Quality', fontsize = 20)
    plt.ylabel(feature.title(), fontsize = 20)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    
plt.subplot(221)
create_boxplot('fixed acidity')

plt.subplot(222)
create_boxplot('volatile acidity')

plt.subplot(223)
create_boxplot('citric acid')

plt.subplot(224)
create_boxplot('residual sugar')

**Observations**:
> - `Fixed Acidity`: The box plot for fixed acidity has approximately the same median for different qualities of wine. The outliers are the highest for wines with a quality of **5**.
> - `Volatile Acidity`: As the quality of the wine increases we can observe that the median values of the volatile acidity of the wine **decreases**.
> - `Citric Acid`: As the quality of the wine increases we can observe that the median values of the citric acid of the wine **increases**. This is exactly **opposite** to the observations we obtained while analysing the `volatile acidity` of the wine.
> - `Residual Sugar`: The residual sugar has almost the **same** median values for different qualities of wine. Wine that has a quality of **5** or **6** has the highest number of outliers.


A violin plot is a hybrid of a box plot and a kernel density plot, which shows peaks in the data. It is used to visualize the distribution of numerical data. Unlike a box plot that can only show summary statistics, violin plots depict summary statistics and the density of each variable. On each side of the gray line is a kernel density estimation to show the distribution shape of the data. Wider sections of the violin plot represent a higher probability that members of the population will take on the given value, the skinnier sections represent a lower probability. This is what a typical violin plot would look like:

<img src = "https://www.researchgate.net/profile/Jonathan-Chambers-3/publication/329035470/figure/fig15/AS:695026912870412@1542718737802/Explanation-of-Violin-plot-Densities-are-estimated-using-a-Gaussian-kernel-density.png" width = 450>

In [ ]:
plt.figure(figsize = (30, 30))

def create_violinplot(feature):
    sns.violinplot(data = df, x = df['quality'], y = feature)
    plt.title('Violin Plot for ' + feature.title(), fontsize = 25)
    plt.xlabel('Quality', fontsize = 20)
    plt.ylabel(feature.title(), fontsize = 20)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    
plt.subplot(221)
create_violinplot('chlorides')

plt.subplot(222)
create_violinplot('free sulfur dioxide')

plt.subplot(223)
create_violinplot('total sulfur dioxide')

plt.subplot(224)
create_violinplot('density')

**Observations**:
> - `Chlorides`: The median values for chlorides is the **same** for different types of wine qualities.
> - `Free Sulfur Dioxide`: The median value for free sulfur dioxide is the highest for wine with a quality of **5**.
> - `Total Sulfur Dioxide`: The **highest IQR** for total sulpfur dioxide belongs to wine with a quality of 5.
> - `Density`: Wine with a quality of 3 has the **highest median value** for density.

The style of **boxenplots** was originally named a `letter value` plot because it shows a large number of quantiles that are defined as `letter values`. It is similar to a box plot in plotting a nonparametric representation of a distribution in which all features correspond to actual observations. By plotting more quantiles, it provides more information about the shape of the distribution, particularly in the tails.

In [ ]:
plt.figure(figsize = (30, 30))

def create_boxenplot(feature):
    sns.boxenplot(data = df, x = df['quality'], y = feature)
    plt.title('Boxenplot for ' + feature.title(), fontsize = 25)
    plt.xlabel('Quality', fontsize = 20)
    plt.ylabel(feature.title(), fontsize = 20)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    
plt.subplot(221)
create_boxenplot('pH')

plt.subplot(222)
create_boxenplot('sulphates')

**Observations**:
> - `Ph`: The median values of the `ph` **decrease** as the quality of wine increases. Wines with a quality of 5 and 6 have longer heads and tails than the others.
> - `Sulphates`: The median value of the `sulphates` **increases** with the increase in the quality of wine. Similar to the `ph`, wines with a quality of 5 and  have longer heads and tails than the rest.

In [ ]:
plt.figure(figsize = (30, 30))


def create_scatterplot(feature1, feature2):
    sns.scatterplot(data = df, x = feature1, y = feature2, hue = df['quality'], 
                    palette = [custom_colors[-1], custom_colors[-2], custom_colors[-3], 
                               custom_colors[-4], custom_colors[-5] ,custom_colors[-6]])
    plt.title(feature1.title() + ' vs ' + feature2.title(), fontsize = 25)
    plt.legend(fontsize = 15)
    plt.xlabel(feature1.title(), fontsize = 20)
    plt.ylabel(feature2.title(), fontsize = 20)
    plt.xticks(fontsize = 16)
    plt.yticks(fontsize = 16)
    
plt.subplot(221)
create_scatterplot('pH', 'fixed acidity')

plt.subplot(222)
create_scatterplot('free sulfur dioxide', 'total sulfur dioxide')

plt.subplot(223)
create_scatterplot('alcohol', 'density')

plt.subplot(224)
create_scatterplot('chlorides', 'free sulfur dioxide')

plt.show()

**Observations**: 
> - `pH` vs `fixed acidity`: The exists a **strong negative correlation** between the features of `pH` and `fixed acidity`.
> - `free sulfur dioxide` vs `total sulfur dioxide`: There seems to be a **postive correlation** between these two features of the dataset.
> - `alcohol` vs `density`: There exists a **negative correlation** between these two features.
> - `chlorides vs free sulfur dioxide`: **No distinct correlation** can be observed from the scatterplot of these two features.

In [ ]:
sns.pairplot(data = df, hue = 'quality', palette = [custom_colors[0], custom_colors[1], custom_colors[2], 
                                                    custom_colors[3], custom_colors[4] ,custom_colors[5]])
plt.show()

A **pairplot** plots a pairwise relationships in a dataset. The pairplot function creates a grid of Axes such that each variable in data will by shared in the Y-axis across a single row and in the X-axis across a single column.

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Checking for Correlation</div>

In [ ]:
plt.figure(figsize = (20, 20))
sns.heatmap(df.corr(), cmap = 'Reds', square = True, annot = True, annot_kws = {'size': 16},
           cbar_kws = {'shrink': 0.80})
plt.title("Visualizing Correlations", size = 25)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.show()

To correct the **class imbalance** in the dataset, we make use of binning. We divide the dataset into `bad` and `good` quality wine, such that the number of samples for **bad** quality wine is almost equal to the number of samples for good quality wine.

In [ ]:
df['quality'] = pd.cut(df['quality'], bins = [1, 5, 10], labels = ['bad', 'good'])

plt.figure(figsize = (15, 10))
ax = sns.countplot(df['quality'], palette = [custom_colors[1], custom_colors[-3]])
bbox_args = dict(boxstyle = 'round', fc = '0.9')
for p in ax.patches:
        ax.annotate('{:.0f} = {:.2f}%'.format(p.get_height(), (p.get_height() / len(df['quality'])) * 100), (p.get_x() + 0.3, p.get_height() + 13), 
                   color = 'black',
                   bbox = bbox_args,
                   fontsize = 15)

plt.title('Quality Count for Red Wine', fontsize = 25)
plt.xlabel('Quality', fontsize = 20)
plt.ylabel('Count', fontsize = 20)
plt.xticks(fontsize = 13)
plt.yticks(fontsize = 13)
plt.show()

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Encoding the Labels</div>

In [ ]:
label_encoder = LabelEncoder()
df['quality'] = label_encoder.fit_transform(df['quality'])
df['quality'].value_counts()

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Scaling the Data</div>
StandardScaler standardizes a feature by subtracting the mean and then scaling it to unit variance.

<img src = "https://miro.medium.com/max/971/1*Nlgc_wq2b-VfdawWX9MLWA.png" width = 250>

In [ ]:
scaler = StandardScaler()
features = [features for features in df.columns if df[features].dtype != int]
df[features] = scaler.fit_transform(df[features])
df

In [ ]:
X = df.drop('quality', axis = 1)
y = df['quality']
print(X, '\n\n\n', y)

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Train-Test Split</div>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Training ML Models</div>

In [ ]:
algo_name = []
accuracy = []

def display_results_and_graphs(algorithm_name, model): 
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_model = model.score(X_test, y_test)
    
    algo_name.append(algorithm_name)
    accuracy.append(acc_model)
    
    print(f'======For {algorithm_name}======')
    print('Training Accuracy: {}%\nTesting Accuracy: {}%\nF1 Score: {}'.
          format((model.score(X_train, y_train) * 100), 
                 model.score(X_test, y_test) * 100, 
                 f1_score(y_test, y_pred)))
    print('\n')
    
    fig, axes = plt.subplots(1, 2, figsize = (15, 8))
    
    fig.suptitle('Graphs for ' + algorithm_name, fontsize = 25)
    
    sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, 
                cmap = 'Reds', annot_kws = {'size': 15}, 
                square = True, fmt = '.0f',
                ax = axes[0])
    axes[0].set_title('Confusion Matrix', fontsize = 20)
    
    fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    sns.lineplot(fpr, tpr, ax = axes[1], color = 'red')
    axes[1].set_title('ROC Curve (' + str(round(roc_auc, 3)) + ')', fontsize = 20)
    axes[1].plot([0, 1], [0, 1,], 'b--')
    plt.show()
    
display_results_and_graphs('Logistic Regression', LogisticRegression())
display_results_and_graphs('K Nearest Neighbors', KNeighborsClassifier(n_neighbors = 13))
display_results_and_graphs('Support Vector Classifier', SVC())
display_results_and_graphs('Decision Tree Classifier', DecisionTreeClassifier(random_state = 0))
display_results_and_graphs('Random Forest Classifer', RandomForestClassifier(random_state = 0))
display_results_and_graphs('Gradient Boosting Classifier', GradientBoostingClassifier(random_state = 0))
display_results_and_graphs('Ada Boost Classifier', AdaBoostClassifier(random_state = 0))
display_results_and_graphs('Cat Boost Classifier', CatBoostClassifier(verbose = 0))
display_results_and_graphs('XGBoost Classifier', XGBClassifier())
display_results_and_graphs('Light Gradient Boosting Machine', lgbm.LGBMClassifier())

In [ ]:
model_comparison = {}

for k, v in zip(algo_name, accuracy):
    model_comparison.update({k: v * 100})

model_comparison = dict(sorted(model_comparison.items(), key = lambda x: x[1], reverse = True))
models = list(model_comparison.keys())
accuracy = list(model_comparison.values())

plt.figure(figsize = (15, 10))
sns.barplot(x = accuracy, y = models)
plt.title('Model Comparison', fontsize = 25)
plt.xlabel('Accuracy', fontsize = 20)
plt.ylabel('Models Used', fontsize = 20)
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.show()

**Observations** We see that ***Random Forest*** gives us the best results with an accuracy of 82.8125%. ***LGBM*** and ***XGBoost*** provide the second and third best results respectively. To improve the accuracy we'll be performing hyperparameter optimization using **Optuna**.

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Hyperparameter Optimization Using Optuna</div>

<img src = "https://raw.githubusercontent.com/optuna/optuna/master/docs/image/optuna-logo.png" width = 350>

**Optuna** is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It features an imperative, ***define-by-run*** style user API. The code written with Optuna enjoys high modularity, allowing the user to dynamically construct search spaces for the hyperparameters.

***Note***: Even though Random Forest gave the best default accuracy, we'll be optimizing the hyperparamters of XGBoost, as it gives better results than an optimized version of Random Forest.

In [ ]:
def objective(trial):
    param_grid = {
        'tree_method': 'gpu_hist', 
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 50),
        'max_depth': trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13, 15, 17]),
        'random_state': 0,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    model = XGBClassifier(**param_grid)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_model = model.score(X_test, y_test)
    
    return acc_model

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 300)

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best Parameters:', study.best_trial.params)
print('Improvement in XGBClassifier Accuracy: {}%'.format((study.best_trial.value * 100) - model_comparison['XGBoost Classifier']))

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Visualizing the Results of Hyperparameter Optimization</div>

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_contour(study,
                                  params = ['alpha',
                                           'learning_rate',
                                           'max_depth',
                                           'n_estimators']
                                 )

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_edf(study)

In [ ]:
X_train = X_train.drop(columns = ['free sulfur dioxide'], axis = 1)
temp = np.array(X_train)
X_train_nn = temp.reshape(-1, 2, 5, 1)
print('New shape of training data:', X_train_nn.shape)

X_test = X_test.drop(columns = ['free sulfur dioxide'], axis = 1)
temp = np.array(X_test)
X_test_nn = temp.reshape(-1, 2, 5, 1)
print('New shape of testing data:', X_test_nn.shape)

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">Prediction Using Neural Networks</div>

In [ ]:
model = Sequential([
    Input(shape = (2, 5, 1)),
    Conv2D(32, 3, padding = 'same', activation = 'relu'),
    Conv2D(32, 3, padding = 'same', activation = 'relu'),
    Conv2D(32, 3, padding = 'same', activation = 'relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding = 'same', activation = 'relu'),
    Conv2D(64, 3, padding = 'same', activation = 'relu'),
    Conv2D(64, 3, padding = 'same', activation = 'relu'),
    Flatten(),
    Dropout(0.2),
    Dense(256, input_shape = (2, 5, 1), activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy',
              optimizer = Adam(),
              metrics = ['acc'])

reduce_lr = ReduceLROnPlateau(monitor = 'acc', patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001)

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
history = model.fit(X_train_nn, y_train,
                    epochs = 100,
                    callbacks = [reduce_lr])

In [ ]:
def model_performance_graphs(classifier):
    
    fig, axes = plt.subplots(1, 2, figsize = (15, 8))

    axes[0].plot(classifier.epoch, classifier.history['acc'], label = 'acc')
    axes[0].set_title('Accuracy vs Epochs', fontsize = 20)
    axes[0].set_xlabel('Epochs', fontsize = 15)
    axes[0].set_ylabel('Accuracy', fontsize = 15)
    axes[0].legend()

    axes[1].plot(classifier.epoch, classifier.history['loss'], label = 'loss')
    axes[1].set_title("Loss Curve",fontsize=18)
    axes[1].set_xlabel("Epochs",fontsize=15)
    axes[1].set_ylabel("Loss",fontsize=15)
    axes[1].legend()

    plt.show()
    
model_performance_graphs(history)

In [ ]:
nn_train_acc = model.evaluate(X_train_nn, y_train)[-1]
nn_test_acc = model.evaluate(X_test_nn, y_test)[-1]
print(nn_train_acc, nn_test_acc)

In [ ]:
y_pred = (model.predict(X_test_nn) > 0.5).astype(int)

print('======For the Neural Network======')
print('Training Accuracy: {}%\nTesting Accuracy: {}%\nF1-Score: {}'.format(nn_train_acc * 100, nn_test_acc * 100, f1_score(y_test, y_pred)))

fig, axes = plt.subplots(1, 2, figsize = (15, 8))
    
fig.suptitle('Graphs for the Neural Network', fontsize = 25)

sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, 
            cmap = 'Reds', annot_kws = {'size': 15}, 
            square = True, fmt = '.0f',
            ax = axes[0])
axes[0].set_title('Confusion Matrix', fontsize = 20)

fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.auc(fpr, tpr)
sns.lineplot(fpr, tpr, ax = axes[1], color = 'red')
axes[1].set_title('ROC Curve (' + str(round(roc_auc, 3)) + ')', fontsize = 20)
axes[1].plot([0, 1], [0, 1,], 'b--')
plt.show()

# <div style = "background-color: #C70000; color:white; border-radius: 15px; padding: 20px; margin: 2px;">References</div>
> - https://www.kaggle.com/code/vishalyo990/prediction-of-quality-of-wine
> - https://www.kaggle.com/code/madhurisivalenka/basic-machine-learning-with-red-wine-quality-data
> - https://www.kaggle.com/code/nkitgupta/feature-engineering-and-feature-selection
> - https://www.analyticsvidhya.com/blog/2020/11/hyperparameter-tuning-using-optuna/